In [27]:
from xml.etree import ElementTree as ET

# читаю xml
with open("oc.xml", mode="r", encoding="utf8") as f:
    doc = ET.parse(f)

all_words = []
all_pos = []

for sent in doc.findall('.//tokens'):
    all_words.append([word.get('text') for word in sent])
    all_pos.append([word.find('.//g').get('v') for word in sent])

print(all_words[:10])
print(all_pos[:10])

[['«', 'Школа', 'злословия', '»', 'учит', 'прикусить', 'язык'], ['Сохранится', 'ли', 'градус', 'дискуссии', 'в', 'новом', 'сезоне', '?'], ['Великолепная', '«', 'Школа', 'злословия', '»', 'вернулась', 'в', 'эфир', 'после', 'летних', 'каникул', 'в', 'новом', 'формате', '.'], ['Потом', 'проект', 'переехал', 'с', '«', 'Культуры', '»', 'на', 'НТВ', '.'], ['Это', 'помимо', 'явных', 'перемен', 'в', 'виде', 'тут', 'же', 'появившихся', 'рекламных', 'блоков', ',', 'отсутствовавших', 'на', '«', 'Культуре', '»', ',', 'позволило', ',', 'с', 'одной', 'стороны', ',', 'расширить', 'круг', 'гостей', ',', 'с', 'другой', '–', 'изменить', 'тон', 'разговора', '.'], ['Народные', 'любимцы'], ['Продлится', 'она', '55', 'дней', ',', 'до', '5', 'ноября', '.'], ['За', 'кота', '–', 'ответишь', '!'], ['Шпаги', ',', 'деньги', ',', 'биржа'], ['Звери', ',', 'деньги', ',', 'наркота']]
[['PNCT', 'NOUN', 'NOUN', 'PNCT', 'VERB', 'INFN', 'NOUN'], ['VERB', 'PRCL', 'NOUN', 'NOUN', 'PREP', 'ADJF', 'NOUN', 'PNCT'], ['ADJF', '

In [28]:
# функция для получения признаков
def get_features_base(current_word, prefix):
    return {
            f'{prefix}[-2:]': current_word[-2:],  # последние две буквы слова
            f'{prefix}letter_case': 'upper' if current_word[0].isupper() else 'lower', # регистр букв
        }

def get_features(sentence, word_index):
    word = sentence[word_index]

    features = get_features_base(word, '')

    if word_index == 0:
        features['START'] = True  # признак начала предложения
    elif word_index == len(sentence) - 1:
        features['END'] = True  # признак конца предложения

    # признаки соседних слов
    if word_index + 1 <= len(sentence) - 1:
        features.update(get_features_base(sentence[word_index + 1][0], 'next-'))
    if word_index - 1 >= 0:
        features.update(get_features_base(sentence[word_index - 1][0], 'prev-'))

    return features

print(get_features(all_words[0], 0))

{'[-2:]': '«', 'letter_case': 'lower', 'START': True, 'next-[-2:]': 'Ш', 'next-letter_case': 'upper'}


In [29]:
# признаки всех слов по предложениям
all_features = [
    [get_features(sentence, ind) for ind in range(len(sentence))]
    for sentence in all_words
]

print(all_features[:2])

[[{'[-2:]': '«', 'letter_case': 'lower', 'START': True, 'next-[-2:]': 'Ш', 'next-letter_case': 'upper'}, {'[-2:]': 'ла', 'letter_case': 'upper', 'next-[-2:]': 'з', 'next-letter_case': 'lower', 'prev-[-2:]': '«', 'prev-letter_case': 'lower'}, {'[-2:]': 'ия', 'letter_case': 'lower', 'next-[-2:]': '»', 'next-letter_case': 'lower', 'prev-[-2:]': 'Ш', 'prev-letter_case': 'upper'}, {'[-2:]': '»', 'letter_case': 'lower', 'next-[-2:]': 'у', 'next-letter_case': 'lower', 'prev-[-2:]': 'з', 'prev-letter_case': 'lower'}, {'[-2:]': 'ит', 'letter_case': 'lower', 'next-[-2:]': 'п', 'next-letter_case': 'lower', 'prev-[-2:]': '»', 'prev-letter_case': 'lower'}, {'[-2:]': 'ть', 'letter_case': 'lower', 'next-[-2:]': 'я', 'next-letter_case': 'lower', 'prev-[-2:]': 'у', 'prev-letter_case': 'lower'}, {'[-2:]': 'ык', 'letter_case': 'lower', 'END': True, 'prev-[-2:]': 'п', 'prev-letter_case': 'lower'}], [{'[-2:]': 'ся', 'letter_case': 'upper', 'START': True, 'next-[-2:]': 'л', 'next-letter_case': 'lower'}, {'[

In [32]:
import math

# тестовый и учебный корпуса
train_set = all_features[:math.floor(len(all_features) * 0.8)]
train_set_pos = all_pos[:math.floor(len(all_pos) * 0.8)]

test_set = all_features[math.floor(len(all_features) * 0.8):]
test_set_pos = all_pos[math.floor(len(all_pos) * 0.8):]

In [33]:
import sklearn_crfsuite

# обучение модельки
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(train_set, train_set_pos)

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

In [71]:
test_sentence = ["Вот", "это", "да", ",", "восстание", "машин", "!"]

test_features = [
    [get_features(test_sentence, ind) for ind in range(len(test_sentence))]
]

crf.predict(test_features)

[['PREP', 'ADJF', 'NOUN', 'PNCT', 'ADJF', 'NOUN', 'PNCT']]

In [37]:
from collections import Counter

# смотрю самые большие и маленькие веса
cnt = Counter(crf.transition_features_)
for p in cnt.most_common()[:5]:
    print(p)

print("---")

for p in cnt.most_common()[-5:]:
    print(p)

(('LATN', 'LATN'), 3.040295)
(('INTJ', 'PNCT'), 2.962795)
(('COMP', 'GRND'), 2.643911)
(('ADJF', 'NOUN'), 2.635062)
(('INTJ', 'NPRO'), 2.632411)
---
(('PREP', 'COMP'), -3.169883)
(('PREP', 'INFN'), -3.455233)
(('PREP', 'PRCL'), -4.263306)
(('VERB', 'VERB'), -4.451824)
(('PREP', 'VERB'), -4.594209)


In [73]:
from sklearn import metrics
import itertools

# определяю точность классификации
# itertools.chain чтобы сделать список плоским
prep_true = list(itertools.chain(*test_set_pos))
prep_pred = list(itertools.chain(*crf.predict(test_set)))

print(metrics.classification_report(prep_true, prep_pred, labels=['NOUN', 'ADJF', 'VERB', 'PREP', 'ADVB'], digits=3))

              precision    recall  f1-score   support

        NOUN      0.871     0.904     0.887      3575
        ADJF      0.789     0.884     0.834       921
        VERB      0.860     0.943     0.900      1069
        PREP      0.923     0.951     0.937       893
        ADVB      0.542     0.472     0.505       271

   micro avg      0.853     0.896     0.874      6729
   macro avg      0.797     0.831     0.812      6729
weighted avg      0.851     0.896     0.873      6729

